In [1]:
# General libraries
import pandas as pd
import numpy as np

from sklearn.base import BaseEstimator, RegressorMixin
from pytorch_tabnet.tab_model import TabNetRegressor as TN
from pytorch_tabnet.augmentations import RegressionSMOTE
from sklearn.model_selection import RandomizedSearchCV

# Pytorch
import torch

# Utils functions
from utils.utils import read_datasets, kfold

In [2]:
x_train, x_test, y_train, y_test = read_datasets(
    'x_train.csv',
    'x_test.csv',
    'y_train.csv',
    'y_test.csv'
)


# convert data to tensor
x_train_tf = torch.tensor(x_train.values.astype(np.float64)) 
y_train_tf = torch.tensor(y_train.values)

x_test_tf = torch.tensor(x_test.values.astype(np.float64)) 
y_test_tf = torch.tensor(y_test.values)

In [3]:
folds = kfold()

In [4]:
class TabNetRegressor(BaseEstimator, RegressorMixin):

    def __init__(self, params={}):
        super(BaseEstimator, self).__init__()
        super(RegressorMixin, self).__init__()
        self.tabnet = TN()
        self.params = params

    def fit(self, X, y=None, params={}):
        if params != {}:
            self.params = params
        self.tabnet.fit(X, y, **self.params)

        print(self.params)
        
        return self
        # return self.tabnet.fit(check_array(X), np.array(y).reshape(-1, 1))
    
    def predict(self, X, y=None):
        preds = self.tabnet.predict(X)
        return preds

    def get_params(self, deep=True):
        return self.params

    def set_params(self, **params):
        for value, key in params.items():
            self.key = value
        return self

In [5]:
max_epochs = 50

aug = RegressionSMOTE(p=0.2)

hyper_params = {
    # 'eval_name':['train'],
    'eval_metric':'rmse',
    'max_epochs':max_epochs,
    'patience':50,
    'batch_size':1024,
    # 'virtual_batch_size':[128],
    'num_workers':0,
    'drop_last':False,
    'augmentations':aug,
}


model = TabNetRegressor(params=hyper_params)
model.fit(X=x_train_tf.numpy(), y=y_train_tf.numpy())

/Users/renan/Library/Python/3.7/lib/python/site-packages/pytorch_tabnet/abstract_model.py:75: UserWarning: Device used : cpu
  warnings.warn(f"Device used : {self.device}")
/Users/renan/Library/Python/3.7/lib/python/site-packages/pytorch_tabnet/abstract_model.py:651: UserWarning: No early stopping will be performed, last training weights will be used.
  warnings.warn(wrn_msg)


epoch 0  | loss: 4381.92529|  0:00:00s
epoch 1  | loss: 4477.31396|  0:00:00s
epoch 2  | loss: 4332.4541|  0:00:00s
epoch 3  | loss: 4335.12695|  0:00:00s
epoch 4  | loss: 4434.16113|  0:00:00s
epoch 5  | loss: 4293.05078|  0:00:00s
epoch 6  | loss: 4295.91406|  0:00:00s
epoch 7  | loss: 4278.11621|  0:00:00s
epoch 8  | loss: 4191.25488|  0:00:00s
epoch 9  | loss: 4115.03027|  0:00:00s
epoch 10 | loss: 4070.47754|  0:00:00s
epoch 11 | loss: 4045.5376|  0:00:00s
epoch 12 | loss: 3953.68701|  0:00:00s
epoch 13 | loss: 3957.26221|  0:00:00s
epoch 14 | loss: 3819.87036|  0:00:00s
epoch 15 | loss: 3798.51685|  0:00:00s
epoch 16 | loss: 3733.75684|  0:00:00s
epoch 17 | loss: 3757.92163|  0:00:00s
epoch 18 | loss: 3634.28394|  0:00:00s
epoch 19 | loss: 3530.59521|  0:00:01s
epoch 20 | loss: 3434.26465|  0:00:01s
epoch 21 | loss: 3369.76074|  0:00:01s
epoch 22 | loss: 3295.0564|  0:00:01s
epoch 23 | loss: 3116.08594|  0:00:01s
epoch 24 | loss: 3115.02222|  0:00:01s
epoch 25 | loss: 3031.24194|

TabNetRegressor(augmentations=<pytorch_tabnet.augmentations.RegressionSMOTE object at 0x129b70390>,
                batch_size=1024, drop_last=False, eval_metric='rmse',
                max_epochs=50, num_workers=0, patience=50)

In [6]:
preds = model.predict(x_test.to_numpy())

print(preds)

[[60.23117 ]
 [54.39115 ]
 [63.43062 ]
 [59.334133]
 [61.88777 ]
 [65.74707 ]
 [42.745224]
 [62.79294 ]
 [56.76098 ]
 [36.524616]
 [62.62951 ]
 [44.430782]
 [72.26399 ]
 [61.281578]
 [72.50192 ]
 [45.298374]
 [74.25104 ]
 [43.960014]
 [69.55649 ]
 [41.976913]
 [61.730988]
 [55.186615]
 [68.0451  ]
 [66.61631 ]
 [49.067535]
 [64.574135]
 [42.630863]
 [62.87755 ]
 [73.68086 ]
 [57.814617]
 [62.49225 ]
 [61.431683]
 [60.23233 ]
 [38.273087]
 [41.26091 ]
 [49.36937 ]
 [65.25845 ]
 [73.62394 ]
 [65.260414]
 [72.49123 ]
 [45.604733]
 [67.07146 ]
 [68.415535]
 [42.206444]
 [40.97438 ]
 [64.9309  ]
 [73.28914 ]
 [68.87916 ]
 [62.074036]
 [54.021873]
 [39.495018]
 [63.926907]
 [55.235878]
 [46.41667 ]
 [63.97428 ]
 [61.892952]
 [60.015575]
 [62.8506  ]
 [62.257042]
 [65.94771 ]
 [55.841896]
 [51.70161 ]
 [62.335552]
 [72.639465]
 [61.421883]
 [45.49151 ]
 [57.70807 ]
 [64.39366 ]
 [49.96189 ]
 [52.943813]
 [65.99477 ]
 [45.053802]
 [63.26612 ]
 [64.35472 ]
 [64.39198 ]
 [61.61534 ]
 [39.393383]

In [7]:
hyper_params_grid = {
    # 'eval_name':['train'],
    'eval_metric':['rmse'],
    'max_epochs':[max_epochs],
    'patience':[50, 100, 150],
    'batch_size':[1024, 64],
    # 'virtual_batch_size':[128],
    'num_workers':[0, 1],
    'drop_last':[False],
    'augmentations':[aug],
}


# Call GridSearchCV()
model_cv = RandomizedSearchCV(
    estimator = TabNetRegressor(),
    param_distributions = hyper_params_grid,
    n_iter=5,
    scoring= 'r2',
    cv = folds,
    verbose = 1,
    return_train_score=True,
    n_jobs = -1,
    # refit = True
)


# Fit the model
best_model = model_cv.fit(X=x_train_tf.numpy(), y=y_train_tf.numpy()) 

Fitting 10 folds for each of 5 candidates, totalling 50 fits


/Users/renan/Library/Python/3.7/lib/python/site-packages/pytorch_tabnet/abstract_model.py:75: UserWarning: Device used : cpu
  warnings.warn(f"Device used : {self.device}")


epoch 0  | loss: 0.0     |  0:00:00s
epoch 1  | loss: 0.0     |  0:00:00s
epoch 2  | loss: 0.0     |  0:00:00s
epoch 3  | loss: 0.0     |  0:00:00s
epoch 4  | loss: 0.0     |  0:00:00s
epoch 5  | loss: 0.0     |  0:00:00s
epoch 6  | loss: 0.0     |  0:00:00s
epoch 7  | loss: 0.0     |  0:00:00s
epoch 8  | loss: 0.0     |  0:00:00s
epoch 9  | loss: 0.0     |  0:00:00s
epoch 10 | loss: 0.0     |  0:00:00s
epoch 11 | loss: 0.0     |  0:00:00s
epoch 12 | loss: 0.0     |  0:00:00s
epoch 13 | loss: 0.0     |  0:00:00s
epoch 14 | loss: 0.0     |  0:00:00s
epoch 15 | loss: 0.0     |  0:00:00s
epoch 16 | loss: 0.0     |  0:00:00s
epoch 17 | loss: 0.0     |  0:00:00s
epoch 18 | loss: 0.0     |  0:00:00s
epoch 19 | loss: 0.0     |  0:00:00s
epoch 20 | loss: 0.0     |  0:00:00s
epoch 21 | loss: 0.0     |  0:00:00s
epoch 22 | loss: 0.0     |  0:00:00s
epoch 23 | loss: 0.0     |  0:00:00s
epoch 24 | loss: 0.0     |  0:00:00s
epoch 25 | loss: 0.0     |  0:00:00s
epoch 26 | loss: 0.0     |  0:00:00s
e

/Users/renan/Library/Python/3.7/lib/python/site-packages/pytorch_tabnet/abstract_model.py:651: UserWarning: No early stopping will be performed, last training weights will be used.
  warnings.warn(wrn_msg)


In [8]:
print(model_cv.best_params_)

{'patience': 50, 'num_workers': 1, 'max_epochs': 50, 'eval_metric': 'rmse', 'drop_last': False, 'batch_size': 1024, 'augmentations': <pytorch_tabnet.augmentations.RegressionSMOTE object at 0x129b70390>}


In [9]:
best_model.get_params()

{'cv': KFold(n_splits=10, random_state=100, shuffle=True),
 'error_score': nan,
 'estimator': TabNetRegressor(),
 'n_iter': 5,
 'n_jobs': -1,
 'param_distributions': {'eval_metric': ['rmse'],
  'max_epochs': [50],
  'patience': [50, 100, 150],
  'batch_size': [1024, 64],
  'num_workers': [0, 1],
  'drop_last': [False],
  'augmentations': [<pytorch_tabnet.augmentations.RegressionSMOTE at 0x129b70390>]},
 'pre_dispatch': '2*n_jobs',
 'random_state': None,
 'refit': True,
 'return_train_score': True,
 'scoring': 'r2',
 'verbose': 1}

In [10]:
max_epochs = 50

aug = RegressionSMOTE(p=0.2)

hyper_params = {
    # 'eval_name':['train'],
    'eval_metric':'rmse',
    'max_epochs':max_epochs,
    'patience':50,
    'batch_size':1024,
    # 'virtual_batch_size':[128],
    'num_workers':0,
    'drop_last':False,
    'augmentations':aug,
}


# Fit the model
best_model = model_cv.fit(X=x_train_tf.numpy(), y=y_train_tf.numpy(), params=hyper_params)

Fitting 10 folds for each of 5 candidates, totalling 50 fits


/Users/renan/Library/Python/3.7/lib/python/site-packages/pytorch_tabnet/abstract_model.py:75: UserWarning: Device used : cpu
  warnings.warn(f"Device used : {self.device}")
/Users/renan/Library/Python/3.7/lib/python/site-packages/pytorch_tabnet/abstract_model.py:651: UserWarning: No early stopping will be performed, last training weights will be used.
  warnings.warn(wrn_msg)


epoch 0  | loss: 4381.92529|  0:00:00s
epoch 1  | loss: 4477.31396|  0:00:00s
epoch 2  | loss: 4332.4541|  0:00:00s
epoch 3  | loss: 4335.12695|  0:00:00s
epoch 4  | loss: 4434.16113|  0:00:00s
epoch 5  | loss: 4293.05078|  0:00:00s
epoch 6  | loss: 4295.91406|  0:00:00s
epoch 7  | loss: 4278.11621|  0:00:00s
epoch 8  | loss: 4191.25488|  0:00:00s
epoch 9  | loss: 4115.03027|  0:00:00s
epoch 10 | loss: 4070.47754|  0:00:00s
epoch 11 | loss: 4045.5376|  0:00:00s
epoch 12 | loss: 3953.68701|  0:00:00s
epoch 13 | loss: 3957.26221|  0:00:00s
epoch 14 | loss: 3819.87036|  0:00:00s
epoch 15 | loss: 3798.51685|  0:00:00s
epoch 16 | loss: 3733.75684|  0:00:00s
epoch 17 | loss: 3757.92163|  0:00:00s
epoch 18 | loss: 3634.28394|  0:00:00s
epoch 19 | loss: 3530.59521|  0:00:00s
epoch 20 | loss: 3434.26465|  0:00:00s
epoch 21 | loss: 3369.76074|  0:00:00s
epoch 22 | loss: 3295.0564|  0:00:00s
epoch 23 | loss: 3116.08594|  0:00:00s
epoch 24 | loss: 3115.02222|  0:00:00s
epoch 25 | loss: 3031.24194|

In [11]:
clf = TN()

aug = RegressionSMOTE(p=0.2)

clf.fit(X_train=x_train_tf.numpy(), y_train=y_train_tf.numpy(),
    # eval_set=[(X_train, y_train), (X_valid, y_valid)],
    # eval_name=['train', 'valid'],
    eval_metric=['rmsle', 'mae', 'rmse', 'mse'],
    max_epochs=max_epochs,
    patience=50,
    batch_size=1024, virtual_batch_size=128,
    num_workers=0,
    drop_last=False,
    augmentations=aug, #aug
)

epoch 0  | loss: 4381.92529|  0:00:00s
epoch 1  | loss: 4477.31396|  0:00:00s
epoch 2  | loss: 4332.4541|  0:00:00s


/Users/renan/Library/Python/3.7/lib/python/site-packages/pytorch_tabnet/abstract_model.py:75: UserWarning: Device used : cpu
  warnings.warn(f"Device used : {self.device}")
/Users/renan/Library/Python/3.7/lib/python/site-packages/pytorch_tabnet/abstract_model.py:651: UserWarning: No early stopping will be performed, last training weights will be used.
  warnings.warn(wrn_msg)


epoch 3  | loss: 4335.12695|  0:00:00s
epoch 4  | loss: 4434.16113|  0:00:00s
epoch 5  | loss: 4293.05078|  0:00:00s
epoch 6  | loss: 4295.91406|  0:00:00s
epoch 7  | loss: 4278.11621|  0:00:00s
epoch 8  | loss: 4191.25488|  0:00:00s
epoch 9  | loss: 4115.03027|  0:00:00s
epoch 10 | loss: 4070.47754|  0:00:00s
epoch 11 | loss: 4045.5376|  0:00:00s
epoch 12 | loss: 3953.68701|  0:00:00s
epoch 13 | loss: 3957.26221|  0:00:00s
epoch 14 | loss: 3819.87036|  0:00:00s
epoch 15 | loss: 3798.51685|  0:00:00s
epoch 16 | loss: 3733.75684|  0:00:00s
epoch 17 | loss: 3757.92163|  0:00:00s
epoch 18 | loss: 3634.28394|  0:00:00s
epoch 19 | loss: 3530.59521|  0:00:00s
epoch 20 | loss: 3434.26465|  0:00:00s
epoch 21 | loss: 3369.76074|  0:00:00s
epoch 22 | loss: 3295.0564|  0:00:00s
epoch 23 | loss: 3116.08594|  0:00:00s
epoch 24 | loss: 3115.02222|  0:00:00s
epoch 25 | loss: 3031.24194|  0:00:00s
epoch 26 | loss: 2838.21899|  0:00:00s
epoch 27 | loss: 2822.99268|  0:00:01s
epoch 28 | loss: 2678.71143

In [35]:
from sklearn.model_selection import ParameterGrid


hyper_params_grid = {
    # 'eval_name':['train'],
    'eval_metric':['rmse'],
    'max_epochs':[max_epochs],
    'patience':[50, 100, 150],
    'batch_size':[1024, 64],
    'virtual_batch_size':[128],
    'num_workers':[0, 1],
    'drop_last':[False],
    'augmentations':[aug],
}


X_train = x_train_tf.numpy()
y_train = y_train_tf.numpy()

# Generate the parameter grid.
param_grid = hyper_params_grid

grid = ParameterGrid(param_grid)

search_results = pd.DataFrame() 
for params in grid:
    # params['n_a'] = params['n_d'] # n_a=n_d always per the paper
    tabnet = TabNetRegressor(params=params)
    #tabnet.set_params(**params)
    tabnet.fit(X=X_train, y=y_train,
               #X_valid=X_train, y_valid=y_train,
            )

    y_prob = tabnet.predict(X=x_test.to_numpy())
    print(y_prob)
    # score = accuracy_score(y_test, y_prob)
    
    # results = pd.DataFrame([params])
    # results['score'] = np.round(score, 3)
    # search_results = search_results.append(results)

/Users/renan/Library/Python/3.7/lib/python/site-packages/pytorch_tabnet/abstract_model.py:75: UserWarning: Device used : cpu
  warnings.warn(f"Device used : {self.device}")
/Users/renan/Library/Python/3.7/lib/python/site-packages/pytorch_tabnet/abstract_model.py:651: UserWarning: No early stopping will be performed, last training weights will be used.
  warnings.warn(wrn_msg)


epoch 0  | loss: 4381.92529|  0:00:00s
epoch 1  | loss: 4477.31396|  0:00:00s
epoch 2  | loss: 4332.4541|  0:00:00s
epoch 3  | loss: 4335.12695|  0:00:00s
epoch 4  | loss: 4434.16113|  0:00:00s
epoch 5  | loss: 4293.05078|  0:00:00s
epoch 6  | loss: 4295.91406|  0:00:00s
epoch 7  | loss: 4278.11621|  0:00:00s
epoch 8  | loss: 4191.25488|  0:00:00s
epoch 9  | loss: 4115.03027|  0:00:00s
epoch 10 | loss: 4070.47754|  0:00:00s
epoch 11 | loss: 4045.5376|  0:00:00s
epoch 12 | loss: 3953.68701|  0:00:00s
epoch 13 | loss: 3957.26221|  0:00:00s
epoch 14 | loss: 3819.87036|  0:00:00s
epoch 15 | loss: 3798.51685|  0:00:00s
epoch 16 | loss: 3733.75684|  0:00:00s
epoch 17 | loss: 3757.92163|  0:00:00s
epoch 18 | loss: 3634.28394|  0:00:00s
epoch 19 | loss: 3530.59521|  0:00:00s
epoch 20 | loss: 3434.26465|  0:00:00s
epoch 21 | loss: 3369.76074|  0:00:00s
epoch 22 | loss: 3295.0564|  0:00:00s
epoch 23 | loss: 3116.08594|  0:00:01s
epoch 24 | loss: 3115.02222|  0:00:01s
epoch 25 | loss: 3031.24194|

/Users/renan/Library/Python/3.7/lib/python/site-packages/pytorch_tabnet/abstract_model.py:75: UserWarning: Device used : cpu
  warnings.warn(f"Device used : {self.device}")
/Users/renan/Library/Python/3.7/lib/python/site-packages/pytorch_tabnet/abstract_model.py:651: UserWarning: No early stopping will be performed, last training weights will be used.
  warnings.warn(wrn_msg)


epoch 1  | loss: 4477.31396|  0:00:00s
epoch 2  | loss: 4332.4541|  0:00:00s
epoch 3  | loss: 4335.12695|  0:00:00s
epoch 4  | loss: 4434.16113|  0:00:00s
epoch 5  | loss: 4293.05078|  0:00:00s
epoch 6  | loss: 4295.91406|  0:00:00s
epoch 7  | loss: 4278.11621|  0:00:00s
epoch 8  | loss: 4191.25488|  0:00:00s
epoch 9  | loss: 4115.03027|  0:00:00s
epoch 10 | loss: 4070.47754|  0:00:00s
epoch 11 | loss: 4045.5376|  0:00:00s
epoch 12 | loss: 3953.68701|  0:00:00s
epoch 13 | loss: 3957.26221|  0:00:00s
epoch 14 | loss: 3819.87036|  0:00:00s
epoch 15 | loss: 3798.51685|  0:00:00s
epoch 16 | loss: 3733.75684|  0:00:00s
epoch 17 | loss: 3757.92163|  0:00:00s
epoch 18 | loss: 3634.28394|  0:00:00s
epoch 19 | loss: 3530.59521|  0:00:01s
epoch 20 | loss: 3434.26465|  0:00:01s
epoch 21 | loss: 3369.76074|  0:00:01s
epoch 22 | loss: 3295.0564|  0:00:01s
epoch 23 | loss: 3116.08594|  0:00:01s
epoch 24 | loss: 3115.02222|  0:00:01s
epoch 25 | loss: 3031.24194|  0:00:01s
epoch 26 | loss: 2838.21899|

/Users/renan/Library/Python/3.7/lib/python/site-packages/pytorch_tabnet/abstract_model.py:75: UserWarning: Device used : cpu
  warnings.warn(f"Device used : {self.device}")
/Users/renan/Library/Python/3.7/lib/python/site-packages/pytorch_tabnet/abstract_model.py:651: UserWarning: No early stopping will be performed, last training weights will be used.
  warnings.warn(wrn_msg)


...

epoch 0  | loss: 4397.90774|  0:00:00s
epoch 1  | loss: 4196.63348|  0:00:00s
epoch 2  | loss: 3984.66448|  0:00:00s
epoch 3  | loss: 3629.802|  0:00:00s
epoch 4  | loss: 3191.74475|  0:00:01s
epoch 5  | loss: 2742.633|  0:00:01s
epoch 6  | loss: 2269.72274|  0:00:01s
epoch 7  | loss: 1797.85631|  0:00:02s
epoch 8  | loss: 1321.59056|  0:00:02s
epoch 9  | loss: 976.41094|  0:00:02s
epoch 10 | loss: 780.61959|  0:00:02s
epoch 11 | loss: 801.84314|  0:00:02s
epoch 12 | loss: 811.36636|  0:00:03s
epoch 13 | loss: 759.73252|  0:00:03s
epoch 14 | loss: 786.42105|  0:00:03s
epoch 15 | loss: 794.60438|  0:00:03s
epoch 16 | loss: 740.71549|  0:00:03s
epoch 17 | loss: 727.40713|  0:00:04s
epoch 18 | loss: 715.36705|  0:00:04s
epoch 19 | loss: 751.5827|  0:00:04s
epoch 20 | loss: 703.35951|  0:00:04s
epoch 21 | loss: 716.41525|  0:00:04s
epoch 22 | loss: 719.94046|  0:00:05s
epoch 23 | loss: 775.68696|  0:00:05s
epoch 24 | loss: 771.61884|  0:00:05s
epoch 25 | loss: 743.52702|  0:00:05s
epoch 26

/Users/renan/Library/Python/3.7/lib/python/site-packages/pytorch_tabnet/abstract_model.py:75: UserWarning: Device used : cpu
  warnings.warn(f"Device used : {self.device}")
/Users/renan/Library/Python/3.7/lib/python/site-packages/pytorch_tabnet/abstract_model.py:651: UserWarning: No early stopping will be performed, last training weights will be used.
  warnings.warn(wrn_msg)


epoch 0  | loss: 4397.90774|  0:00:00s
epoch 1  | loss: 4196.63348|  0:00:00s
epoch 2  | loss: 3984.66448|  0:00:00s
epoch 3  | loss: 3629.802|  0:00:00s
epoch 4  | loss: 3191.74475|  0:00:01s
epoch 5  | loss: 2742.633|  0:00:01s
epoch 6  | loss: 2269.72274|  0:00:01s
epoch 7  | loss: 1797.85631|  0:00:01s
epoch 8  | loss: 1321.59056|  0:00:01s
epoch 9  | loss: 976.41094|  0:00:02s
epoch 10 | loss: 780.61959|  0:00:02s
epoch 11 | loss: 801.84314|  0:00:02s
epoch 12 | loss: 811.36636|  0:00:02s
epoch 13 | loss: 759.73252|  0:00:02s
epoch 14 | loss: 786.42105|  0:00:03s
epoch 15 | loss: 794.60438|  0:00:03s
epoch 16 | loss: 740.71549|  0:00:03s
epoch 17 | loss: 727.40713|  0:00:03s
epoch 18 | loss: 715.36705|  0:00:03s
epoch 19 | loss: 751.5827|  0:00:04s
epoch 20 | loss: 703.35951|  0:00:04s
epoch 21 | loss: 716.41525|  0:00:04s
epoch 22 | loss: 719.94046|  0:00:04s
epoch 23 | loss: 775.68696|  0:00:05s
epoch 24 | loss: 771.61884|  0:00:05s
epoch 25 | loss: 743.52702|  0:00:05s
epoch 26

/Users/renan/Library/Python/3.7/lib/python/site-packages/pytorch_tabnet/abstract_model.py:75: UserWarning: Device used : cpu
  warnings.warn(f"Device used : {self.device}")
/Users/renan/Library/Python/3.7/lib/python/site-packages/pytorch_tabnet/abstract_model.py:651: UserWarning: No early stopping will be performed, last training weights will be used.
  warnings.warn(wrn_msg)


epoch 0  | loss: 4397.90774|  0:00:00s
epoch 1  | loss: 4196.63348|  0:00:00s
epoch 2  | loss: 3984.66448|  0:00:00s
epoch 3  | loss: 3629.802|  0:00:00s
epoch 4  | loss: 3191.74475|  0:00:01s
epoch 5  | loss: 2742.633|  0:00:01s
epoch 6  | loss: 2269.72274|  0:00:01s
epoch 7  | loss: 1797.85631|  0:00:01s
epoch 8  | loss: 1321.59056|  0:00:01s
epoch 9  | loss: 976.41094|  0:00:02s
epoch 10 | loss: 780.61959|  0:00:02s
epoch 11 | loss: 801.84314|  0:00:02s
epoch 12 | loss: 811.36636|  0:00:02s
epoch 13 | loss: 759.73252|  0:00:02s
epoch 14 | loss: 786.42105|  0:00:03s
epoch 15 | loss: 794.60438|  0:00:03s
epoch 16 | loss: 740.71549|  0:00:03s
epoch 17 | loss: 727.40713|  0:00:03s
epoch 18 | loss: 715.36705|  0:00:03s
epoch 19 | loss: 751.5827|  0:00:04s
epoch 20 | loss: 703.35951|  0:00:04s
epoch 21 | loss: 716.41525|  0:00:04s
epoch 22 | loss: 719.94046|  0:00:04s
epoch 23 | loss: 775.68696|  0:00:04s
epoch 24 | loss: 771.61884|  0:00:05s
epoch 25 | loss: 743.52702|  0:00:05s
epoch 26